<a href="https://colab.research.google.com/github/ridhimagoyal/Object-Detection-Basic/blob/main/Pulling_Tweets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install git+https://github.com/tweepy/tweepy.git

  Cloning https://github.com/tweepy/tweepy.git to /tmp/pip-req-build-osysj4oj
  Running command git clone --filter=blob:none --quiet https://github.com/tweepy/tweepy.git /tmp/pip-req-build-osysj4oj
  Resolved https://github.com/tweepy/tweepy.git to commit db28c0e84826485755eb7fcef0c30f75395dff5f
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [7]:
pip install tweepy


In [9]:
# import tweepy
# import pandas as pd

# # Replace with your Bearer Token
# bearer_token = "AAAAAAAAAAAAAAAAAAAAAFrt2QEAAAAANy2XWP%2B9xTUaeCPNsbu3AbpQi%2Bg%3DScPVtPd5u4fEh5xwKRoQREg26fsmP2kFsNOoTsFkXWzMeBbddG"

# # Initialize Tweepy Client
# client = tweepy.Client(bearer_token=bearer_token)

# # Define your keywords
# keywords = ["block", "fight", "strike"]
# query = " OR ".join(keywords) + " -is:retweet -is:reply"  # Exclude retweets and replies

# # Fetch recent tweets
# response = client.search_recent_tweets(
#     query=query,
#     tweet_fields=["created_at", "public_metrics", "author_id", "lang"],
#     max_results=50,  # Max per request is 100
# )

# # Parse results into a DataFrame (if tweets found)
# if response.data:
#     tweets_data = [
#         {
#             "Tweet": tweet.text,
#             "Created At": tweet.created_at,
#             "Author ID": tweet.author_id,
#             "Like Count": tweet.public_metrics["like_count"],
#             "Retweet Count": tweet.public_metrics["retweet_count"],
#             "Language": tweet.lang
#         }
#         for tweet in response.data
#     ]
#     df = pd.DataFrame(tweets_data)
#     print(df)
# else:
#     print("No tweets found matching your keywords.")


                                                Tweet  \
0   RT @BriannaWu: If you’ve noticed a strong down...   
1   Na cos he no dey fight for pitch\nExpose that ...   
2   Just had the craziest day! Spent the morning a...   
3   RT @MuminJamiean: If you respond with 3 replys...   
4   RT @bennyjohnson: Every man needs something to...   
5             @purse_jame48483 @smencimer 🖕and block.   
6   RT @DorianePin: What a win!! 💪\n\nGreat to fig...   
7   Omo 1 girl like that from Sudan dey video call...   
8   RT @emilykschrader: At the site of the missile...   
9   RT @codyrussel70504: Hello friends! \nMeet my ...   
10  @matt_vanswol @nicksortor Because republicans ...   
11  RT @lunartifa: Tifa’s fight in AC will always ...   
12  RT @xupseude: how to make the champion fold (n...   
13  RT @AndresFleurqui1: Girl fight at school http...   
14  RT @JKRadioUpdater: 🚨Report -&gt; Mute -&gt; B...   
15  @Rameshshetty500 @ImDanger143 @LoudAndKlear Ev...   
16  RT @bennyjohnson: Every man

In [12]:
import tweepy
import pandas as pd
from textblob import TextBlob

bearer_token = "AAAAAAAAAAAAAAAAAAAAAFrt2QEAAAAANy2XWP%2B9xTUaeCPNsbu3AbpQi%2Bg%3DScPVtPd5u4fEh5xwKRoQREg26fsmP2kFsNOoTsFkXWzMeBbddG"

client = tweepy.Client(bearer_token=bearer_token)

keywords = [
    "protest", "strike", "shutdown", "roadblock", "blockade",
    "rail roko", "andolan", "dharna", "rasta roko", "traffic jam"
]

target_regions = ["Delhi", "Mumbai","Hyderabad"]
query = "(" + " OR ".join(keywords) + ")" + " (" + " OR ".join(target_regions) + ")" + " -is:retweet -is:reply"

response = client.search_recent_tweets(
    query=query,
    tweet_fields=["created_at", "public_metrics", "author_id", "lang"],
    max_results=50,
)

def classify_tweet(text):
    text_lower = text.lower()
    if "protest" in text_lower or "andolan" in text_lower:
        event_type = "Protest"
    elif "strike" in text_lower or "dharna" in text_lower:
        event_type = "Strike"
    elif "shutdown" in text_lower:
        event_type = "Shutdown"
    elif any(word in text_lower for word in ["traffic jam", "roadblock", "rasta roko"]):
        event_type = "Blockage"
    else:
        event_type = "Other"

    # Severity analysis using sentiment
    polarity = TextBlob(text).sentiment.polarity
    severity = "High" if polarity < -0.3 else "Medium" if polarity < 0 else "Low"

    return event_type, severity

# Store tweets with classification
tweets_data = []
if response.data:
    for tweet in response.data:
        event_type, severity = classify_tweet(tweet.text)
        tweets_data.append({
            "Tweet": tweet.text,
            "Created At": tweet.created_at,
            "Author ID": tweet.author_id,
            "Like Count": tweet.public_metrics["like_count"],
            "Retweet Count": tweet.public_metrics["retweet_count"],
            "Language": tweet.lang,
            "Event Type": event_type,
            "Severity": severity
        })

    df = pd.DataFrame(tweets_data)
    print(df)
else:
    print("No relevant tweets found.")


TooManyRequests: 429 Too Many Requests
Too Many Requests

In [ ]:
supplier_regions = {
    "SupplierA": "Delhi",
    "SupplierB": "Mumbai",
    "SupplierC": "Hyderabad"
}
severity_score = {
    "High": 3,
    "Medium": 2,
    "Low": 1
}

# Assuming `df` is your DataFrame from the Twitter fetch
def calculate_risk(df, supplier_regions):
    severity_score = {"High": 3, "Medium": 2, "Low": 1}
    region_scores = {region: 0 for region in set(supplier_regions.values())}

    for _, row in df.iterrows():
        for region in region_scores:
            if region.lower() in row["Tweet"].lower():
                region_scores[region] += severity_score[row["Severity"]]

    supplier_risk = {}
    for supplier, region in supplier_regions.items():
        score = region_scores.get(region, 0)
        if score >= 9:
            risk = "High Risk"
        elif score >= 4:
            risk = "Medium Risk"
        else:
            risk = "Low Risk"
        supplier_risk[supplier] = {"Region": region, "Score": score, "Risk Level": risk}

    return supplier_risk


In [4]:
import tweepy
import pandas as pd

consumer_key = "R6JP5NRFzWYCB2uEHk1SFoRFb" #Your API/Consumer key
consumer_secret = "4kmuzAfSqBw8HCzuRWo9UBKVsDwasPfGZa74Aby1jZ1LvqpXxP" #Your API/Consumer Secret Key
access_token = "1711757990796341248-kQFj3kfMdSL0MyYdWlEYrzbGsUZmgz"   #Your Access token key
access_token_secret = "H6ntYMFDm46Gslp7MzfV3VSqkZxjg2qqadiVykomtFqx7" #Your Access token Secret key


Status Failed On, 403 Forbidden
453 - You currently have access to a subset of X API V2 endpoints and limited v1.1 endpoints (e.g. media post, oauth) only. If you need access to this endpoint, you may need a different access level. You can learn more here: https://developer.x.com/en/portal/product


In [6]:
tweets_df

NameError: name 'tweets_df' is not defined

In [ ]:
tweets[0]

Status(_api=<tweepy.api.API object at 0x7f2c587c0b80>, _json={'created_at': 'Fri Dec 23 04:38:40 +0000 2022', 'id': 1606147252468211714, 'id_str': '1606147252468211714', 'full_text': 'Once again, the English are criticising everyone in their world cup defeat except the guy who missed a penalty that would have taken the game to extra time. They blame the ref, FIFA, Rashford, climate change but not Mr England (Kane) who hoofed that ball into the stands.', 'truncated': False, 'display_text_range': [0, 271], 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [], 'urls': []}, 'metadata': {'iso_language_code': 'en', 'result_type': 'recent'}, 'source': '<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 1267956633830862850, 'id_str': '1267956633830862850', 'name': 'Muswema M

In [ ]:
from google.colab import files
tweets_df.to_csv('tweets.csv')
files.download('tweets.csv')